In [ ]:
import pandas, os, sys, re, time, collections, Bio, pprint

sys.path.append('/Users/dp/pma/')

import sameRiver
#import sameRiver.biotypeUtils # For biotypeLookupFileMaker

from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from pprint import pprint as pp
from functools import reduce

# Study descriptions obtained from:
# https://www.cbioportal.org/webservice.do?cmd=getCancerTypes

In [ ]:
our_rbps = ['FUBP1', 'HNRNPCL1', 'RPL5', 'TDRKH', 'EEF1B2', 'PCBP1', 'SF3B1', 'PABPC4L', 
            'YTHDC2', 'BCLAF1', 'KHDRBS2', 'RARS2', 'A1CF', 'DDX50', 'UPF2', 'DCP1B', 'NUFIP1', 
            'DICER1', 'NOVA1', 'CNOT1', 'RBFOX1', 'KPNB1', 'SRSF2', 'CRNKL1', 'RBM39', 'RBM11', 
            'U2AF1', 'DDX3X', 'EIF1AX', 'RPL10', 'RQCD1'] 
our_rbps_as_str = '+'.join(our_rbps)


In [ ]:
df = pandas.read_csv('uniprot_id_to_gene_name.tab', sep='\t', index_col=False)
df.columns = ['Uniprot', 'Gene']
uniprot_to_gene_name = dict(zip([x.upper() for x in df.Uniprot], [x.upper() for x in df.Gene]))
amigo = pandas.read_csv('listsOfRbps/rna_binding_genes_from_amigo.txt', sep='\t', comment='#')['Gene name'].tolist()

missing_rbps = ['TDRKH', 'CNOT9', 'KHDRBS2', 'UPF2']
missing_rbps.extend(our_rbps)
for rbp in missing_rbps:
    if rbp not in amigo:
        amigo.append(rbp)
        
census = pandas.read_excel('listsOfRbps/rbp_table_from_rbp_census_nrg3818_s3.xls')['gene name'].tolist()
census = set(census) - set([''])
census = census - set(amigo)
census = list(census)

        
not_actually_rbps = [
#    'BRCA1', 'CDKN2A', 'JUN'
]
amigo_as_str = '+'.join(amigo)
census_as_str = '+'.join(census)
our_rbps_as_str = '+'.join(our_rbps)

chunk_size = 100
amigo_chunks = []
for n in range(0, len(amigo), 100):
    right_edge = min([n+100, len(amigo)])
    amigo_chunks.append('+'.join(amigo[n:right_edge]))

census_chunks = []
for n in range(0, len(census), 100):
    right_edge = min([n+100, len(census)])
    census_chunks.append('+'.join(census[n:right_edge]))
print(census_as_str[:30])
print(len(census))
#print(amigo_chunks)

In [ ]:
import re
m = re.search('MYC', amigo_as_str)
print(m)
for chunk in amigo_chunks:
    m = re.search('MYC', chunk)
    print(m)

In [ ]:
df = pandas.read_csv('all_TCGA_data/mutations.txt', sep='\t')
study_ids = set(df['STUDY_ID'].tolist())
print(len(study_ids))
print(study_ids)

In [ ]:
import requests
import json
for study_id in study_ids:
    url = f'https://www.cbioportal.org/api/studies/{study_id}/'
    url += 'patients?projection=DETAILED&pageSize=10000000&pageNumber=0' 
    r = requests.get(url)
    f = open(f'all_TCGA_data/patient_info/{study_id}.txt', 'w')
    f.write(r.text)
    f.close()
    time.sleep(5)

In [ ]:


#r = requests.get(url)
#for row in r.json():
#    print(row)
f = open('all_TCGA_data/patient_info/mixed_allen_2018.txt')
data = json.load(f)
f.close()

print(data)

In [ ]:
import requests
import json

#amigo_chunks = [our_rbps_as_str]
url = 'http://www.cbioportal.org/webservice.do?cmd=getCaseLists&cancer_study_id=gbm_tcga'
url = 'http://www.cbioportal.org/webservice.do?cmd=getProfileData&case_set_id=gbm_tcga_all&genetic_profile_id=gbm_tcga_mutations&gene_list=BRCA1+BRCA2+TP53'
url = 'http://www.cbioportal.org/webservice.do?cmd=getProfileData&cancer_study_id=mixed_pipseq_2017&case_set_id=mixed_pipseq_2017_all&genetic_profile_id=mixed_pipseq_2017_mutations&gene_list=' + amigo_chunks[0]


amigo_chunks = census_chunks

done_study_ids = set()
for study_id in study_ids:
    outputs = []
    print('\n({}/{})'.format(len(done_study_ids)+1, len(study_ids)), study_id)
    fname = 'all_TCGA_data/Jan2020_census_api_outputs/census_{}.txt'.format(study_id)
    if os.path.exists(fname):
        print("Skipping {} (file exists).".format(study_id))
        done_study_ids.add(study_id)
        continue
    f = open(fname, 'w')
    
    for n, amigo_chunk in enumerate(amigo_chunks):
        print('Chunk {}/{}'.format(n+1, len(amigo_chunks)), end=', ')
        url = 'http://www.cbioportal.org/webservice.do?cmd=getProfileData'
        url += '&cancer_study_id={std}&case_set_id={std}_all&genetic_profile_id={std}_mutations'.format(std=study_id)
        url += '&gene_list=' + amigo_chunk
        #url = 'http://www.cbioportal.org/webservice.do?cmd=getGeneticProfiles&cancer_study_id=mixed_pipseq_2017'
        if 'MYC' in amigo_chunk:
            print("MYC in chunk:", amigo_chunk)
        r = requests.get(url)
        f.write(r.text)
#        lines = r.text.split('\n')
#        outputs.append(r.text)
        time.sleep(5)
    f.close()
    done_study_ids.add(study_id)
    time.sleep(10)
#lines = r.text.split('\n')
#for li in lines:
#    print(li)
#print(lines[:3]) 
